In [1]:
import os
import numpy as np
import cv2 as cv
import glob
from pathlib import Path
print(cv.__version__)

bmp_dir = "./my_imgs/pic1/bmp"
jpg_dir = "./my_imgs/pic1/jpg"

if not Path(jpg_dir).is_dir():
    os.mkdir(jpg_dir)

# bmp格式图像转为jpg
filelists = os.listdir(bmp_dir)
for f in filelists:
    img = cv.imread(os.path.join(bmp_dir, f), -1)
    newName = f.replace('.bmp', '.jpg')
    cv.imwrite(os.path.join(jpg_dir, newName), img)

4.6.0


In [2]:
# 相机标定
def calib(inter_corner_shape, size_per_grid, img_dir, img_type):
    print(inter_corner_shape)  # cv::Size(columns，rows)
    w, h = inter_corner_shape

    # cp_int: 世界坐标系中int格式的角点序号坐标，如 (0,0,0), (1,0,0), (2,0,0) ....,(10,7,0).
    cp_int = np.zeros((w*h,3), np.float32)
    cp_int[:,0:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)
    # cp_world: 世界坐标系中的角点坐标
    cp_world = cp_int * size_per_grid

    obj_points = []  # 空间坐标系中的点
    img_points = []  # 像素坐标系中的点
    images = glob.glob(img_dir + os.sep + '**.' + img_type)  # 查找符合规则的文件路径名

    draw_save_dir = img_dir.replace(img_type, 'corner')
    if not Path(draw_save_dir).is_dir():
        os.mkdir(draw_save_dir)

    for i, fname in enumerate(images):
        img = cv.imread(fname)
        gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        # 查找角点，cp_img: 像素坐标系中的角点坐标
        ret, cp_img = cv.findChessboardCorners(gray_img, inter_corner_shape, None)
        if ret == True:
            obj_points.append(cp_world)
            img_points.append(cp_img)
            draw_img = cv.drawChessboardCorners(img, inter_corner_shape, cp_img, ret)
            cv.imwrite(os.path.join(draw_save_dir, str(i+1)+'.jpg'), draw_img)
        else:
            raise RuntimeError("Find Chessboard Corners Error !")

    # 相机标定
    ret, mat_inter, coff_dis, v_rot, v_trans = cv.calibrateCamera(obj_points, img_points, gray_img.shape[::-1], None, None, flags=(cv.CALIB_ZERO_TANGENT_DIST+cv.CALIB_FIX_K3))
    print("Ret:", ret)
    print("Internal matrix:\n", mat_inter)
    print("Distortion Cofficients:\n", coff_dis)
    print("Rotation vectors:\n", v_rot[5])
    print("Translation vectors:\n", v_trans[5])
    # 计算重投影误差
    total_error = 0
    for i in range(len(obj_points)):
        # 世界坐标系中的点重投影到像素坐标系中
        img_points_repro, _ = cv.projectPoints(obj_points[i], v_rot[i], v_trans[i], mat_inter, coff_dis)
        error = cv.norm(img_points[i], img_points_repro, cv.NORM_L2) / len(img_points_repro)
        total_error += error
    print("Average Error of Reproject:", total_error/len(obj_points))

    return mat_inter, coff_dis

inter_corner_shape = (6, 8)
size_per_grid = 0.03
img_type = 'jpg'
mat_inter, coff_dis = calib(inter_corner_shape, size_per_grid, jpg_dir, img_type);


(6, 8)
Ret: 0.5694511814872326
Internal matrix:
 [[862.47933407   0.         655.93105476]
 [  0.         859.28839578 511.04247192]
 [  0.           0.           1.        ]]
Distortion Cofficients:
 [[-0.09334898  0.06179651  0.          0.          0.        ]]
Rotation vectors:
 [[-0.11434551]
 [ 0.11502402]
 [-0.07087439]]
Translation vectors:
 [[-0.09565525]
 [-0.1648946 ]
 [ 0.55810831]]
Average Error of Reproject: 0.052574249817994946


In [3]:
# 计算两张图片之间的单应性矩阵
def findH2(dst, src, inter_corner_shape, img_dir, img_type):
    images = glob.glob(img_dir + os.sep + '**.' + img_type)  # 查找符合规则的文件路径名

    img_0 = cv.imread(images[src])
    gray_img_0 = cv.cvtColor(img_0, cv.COLOR_BGR2GRAY)
    ret_0, corners_0 = cv.findChessboardCorners(gray_img_0, inter_corner_shape, None)

    img = cv.imread(images[dst])
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray_img, inter_corner_shape, None)

    if ret and ret_0:
        H, mask = cv.findHomography(corners_0, corners, cv.RANSAC)  # mask: 内点掩码
        print(H)
    else:
        raise RuntimeError("Find Chessboard Corners Error !")

    # 验证单应性变换方向
    Reproj_corners = cv.perspectiveTransform(corners_0, H)  #! dst = H src
    total_error = 0
    for i in range(len(corners)):
        error = cv.norm(corners[i], Reproj_corners[i], cv.NORM_L2) ** 2
        total_error += error
    total_error /= 2
    print("Average Error of Reproject:", total_error)

    return H

src_img_idx = 0
dst_img_idx = 7
H = findH2(dst_img_idx, src_img_idx, inter_corner_shape, jpg_dir, img_type);

[[ 1.36853223e+00 -8.44297222e-02 -9.05552359e+01]
 [ 7.99679834e-02  1.09797540e+00 -6.47323312e+01]
 [ 2.29615676e-04 -2.26052358e-04  1.00000000e+00]]
Average Error of Reproject: 1.4365618175361305


In [4]:
# 由H矩阵求解旋转矩阵，筛选正确 R 和 t
num, Rs, Ts, Ns  = cv.decomposeHomographyMat(H, mat_inter)
K = mat_inter
K_inv = np.linalg.inv(K)
z0 = np.array([[0], [0], [1]])  # z轴单位向量

flags = np.ones(num, dtype=np.int16)
for i in range(num):
    R, T, N = Rs[i], Ts[i], Ns[i]
    # 法向量和 z 轴夹角的余弦值即为 N[2]
    if (abs(N[2]) < 0.85):
        flags[i] = 0
    # d
    d = (T @ N.T) / (R - K_inv @ H @ K)
print(flags)


[1 1 0 0]


In [5]:
# PnP问题求解相机位姿
def usePnP(idx, mat_inter, coff_dis, inter_corner_shape, size_per_grid, img_dir, img_type):
    w, h = inter_corner_shape

    # cp_int: 世界坐标系中int格式的角点序号坐标，如 (0,0,0), (1,0,0), (2,0,0) ....,(10,7,0).
    cp_int = np.zeros((w*h,3), np.float32)
    cp_int[:,0:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)
    # cp_world: 世界坐标系中的角点坐标
    cp_world = cp_int * size_per_grid

    # cp_img: 像素坐标系中的角点坐标
    images = glob.glob(img_dir + os.sep + '**.' + img_type)  # 查找符合规则的文件路径名
    img = cv.imread(images[idx])
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, cp_img = cv.findChessboardCorners(gray_img, inter_corner_shape, None)

    # 估计相机位姿
    _, v_rot, T = cv.solvePnP(cp_world, cp_img, mat_inter, coff_dis)
    print("Rotation vectors:\n", v_rot)
    print("Translation vectors:\n", T)

usePnP(5, mat_inter, coff_dis, inter_corner_shape, size_per_grid, jpg_dir, img_type)

Rotation vectors:
 [[-0.11434551]
 [ 0.11502402]
 [-0.07087439]]
Translation vectors:
 [[-0.09565525]
 [-0.1648946 ]
 [ 0.55810831]]
